## Code Hist.

 - CODE : KIER Data_순시 사용량 계산 (전처리 X)  
 - DESC  
    &ensp; : int_Domain 코드의 변경에 따라,  
    &emsp; 동/층/세대별로 분리된 에너지 사용량 데이터(ACCU)로부터  
    &emsp; 순시사용량을 산출  
  - DATE  
    &ensp; 2024-07-26 Created : "M02-01_Data_02_Raw To House_Separation By Column"로부터
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&ensp; Src 분리  
    &ensp; 2024-07-26 코드 개선 : 개별 CSV 저장 방식으로부터 통합 저장으로 변경  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; (용량 절감 목적)  
    &ensp; 2024-07-26 코드 개선 : 에너지 사용량 음수값 부분 제거 Code 추가  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&ensp; - 1안 : 음수값을 단순 소거  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&ensp; - 2안 : 음수값을 양수값으로 전환 (이후 IQR 제거에서 소거 하는 방식)  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&ensp; -> 이 중 1안의 Code 추가

## Code

### 01. Init

#### 01-01. Module Import

In [100]:
#region Import_Basic Module
## Basic
import os, sys, warnings
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.path.dirname(os.path.abspath('./__file__'))
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('./__file__'))))
warnings.filterwarnings('ignore')

import numpy as np, pandas as pd
from pandas import DataFrame, Series
pd.options.display.float_format = '{:.10f}'.format

import math, random

## Datetime
import time, datetime as dt
from datetime import datetime, date, timedelta

## glob
import glob, requests, json
from glob import glob

## 시각화
import matplotlib.pyplot as plt, seaborn as sns
# %matplotlib inline
plt.rcParams['figure.figsize'] = [10, 8]

from scipy import stats

## Split, 정규화
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# K-Means 알고리즘
from sklearn.cluster import KMeans, MiniBatchKMeans

# Clustering 알고리즘의 성능 평가 측도
from sklearn import metrics
from sklearn.metrics import homogeneity_score, completeness_score, v_measure_score, adjusted_rand_score, silhouette_score, rand_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.metrics.cluster import contingency_matrix

## For Web
import urllib
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

import tqdm
from tqdm.notebook import tqdm
#endregion Import_Basic Module

In [101]:
## Import_DL
str_tar = "tf"
## For Torch
if str_tar == "torch":
    import torch, torch.nn as nn
    from torch.nn.utils import weight_norm
    print("Torch Imported")
## For TF
elif str_tar == "tf":
    import tensorflow as tf, tensorflow_addons as tfa
    from keras.callbacks import EarlyStopping, ModelCheckpoint
    from keras.models import Sequential, load_model
    from keras_flops import get_flops
    print("Tensorflow Imported")
else:
    print("Error : Cannot be used except for Keywords")
    print(" : torch / tf")

Tensorflow Imported


In [102]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2692405796033011370
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 22395486208
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 11827883784069093123
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9"
 xla_global_id: 416903419]

In [103]:
tf.debugging.set_log_device_placement(True)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # 텐서플로가 첫 번째 GPU만 사용하도록 제한
  try:
    print('Using GPU')
    tf.config.set_visible_devices(gpus[0], 'GPU')
  except RuntimeError as e:
    # 프로그램 시작시에 접근 가능한 장치가 설정되어야만 합니다
    print(e)
else : 
  print('Using CPU')

Using GPU


In [104]:
## Import_Local
from Src_Dev_Common import Data_Datetime as com_date, KMA_Weather as com_KMA, KECO_AirKor as com_KECO, KASI_Holiday as com_Holi, KIER_Usage_M02 as com_KIER_M02

#### 01-02. Config (Directory, Params)

In [105]:
## Init_config
SEED = 42

np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)
os.environ["PYTHONHASHSEED"], os.environ['TF_DETERMINISTIC_OPS'] = str(SEED), "1"

In [106]:
## Define Todate str
str_now_ymd = pd.datetime.now().date()
str_now_y, str_now_m, str_now_d = pd.datetime.now().year, pd.datetime.now().month, pd.datetime.now().day
str_now_hr, str_now_min = pd.datetime.now().hour, pd.datetime.now().minute

print(pd.datetime.now())
print(str(str_now_y) + " / " + str(str_now_m)  + " / " + str(str_now_d))
print(str(str_now_hr) + " : " + str(str_now_min))

2024-07-30 18:25:59.799853
2024 / 7 / 30
18 : 25


### 02. Data Prepare

In [107]:
## Dict_Domain
int_domain = 0

## Domain, ACCU/INST Column
str_domain, str_col_accu, str_col_inst = com_KIER_M02.create_domain_str(int_domain)
## Directory Root
str_dirData, str_dir_raw, str_dir_cleansed, str_dirName_bld, str_dirName_h = com_KIER_M02.create_dir_str(str_domain)

## File
str_fileRaw, str_fileRaw_hList = str('KIER_RAW_' + str_domain + '_H_ID_Adopted.csv'), str('KIER_hList_Common.csv')

print(str(os.listdir(str_dirData)) + "\n")
print(os.listdir(str_dirName_h))

0 : ELEC
['.ipynb_checkpoints', 'BS_CONFIGURATION_202309251452.csv', 'DATE_1M_2023-10-20.csv', 'KIER 전처리 현황_2024-06-25.xlsx', 'KIER_0_Raw', 'KIER_1_Cleansed', 'KIER_2_BLD', 'KIER_3_H_ELEC', 'KIER_3_H_GAS', 'KIER_3_H_HEAT', 'KIER_3_H_HOT_FLOW', 'KIER_3_H_HOT_HEAT', 'KIER_3_H_WATER', 'KIER_ASOS_WEATHER_DAILY_202309251521.csv', 'KIER_ASOS_WEATHER_HOUR_202309251521.csv', 'KIER_DATA_OLD', 'KIER_ETC', 'KIER_hList_Comparison_2024-06-26.xlsx', 'KIER_List_Table_Column_2023-09-25.xlsx', 'KIER_Query_2023-09-25.txt', 'KMA_ASOS_119_2010_2023_1st_to CSV.csv', '[IITP] 데이터 테이블 정리 (공유 원본).docx']

['KIER_ELEC_561-1-1_ACCU_01_Raw.csv', 'KIER_ELEC_561-1-2_ACCU_01_Raw.csv', 'KIER_ELEC_561-1-3_ACCU_01_Raw.csv', 'KIER_ELEC_561-1-4_ACCU_01_Raw.csv', 'KIER_ELEC_561-10-1_ACCU_01_Raw.csv', 'KIER_ELEC_561-10-2_ACCU_01_Raw.csv', 'KIER_ELEC_561-10-3_ACCU_01_Raw.csv', 'KIER_ELEC_561-10-4_ACCU_01_Raw.csv', 'KIER_ELEC_561-11-1_ACCU_01_Raw.csv', 'KIER_ELEC_561-11-2_ACCU_01_Raw.csv', 'KIER_ELEC_561-11-3_ACCU_01_Raw.csv'

#### 02-01. KIER (hList)

In [108]:
## "KIER_01-01_Data_hList.ipynb"로부터 만들어진 Bld/F/H List
df_kier_hList = pd.read_csv(str_dir_raw + str_fileRaw_hList, index_col = 0)
# print(df_kier_hList.shape, ' /// ', df_kier_hList.columns)
# df_kier_hList

#### 02-01. KIER (Energy Usage : ACCU Combined)

In [109]:
## Load Intergrated ACCU Usage
str_file_accu = 'KIER_' + str_domain + '_ACCU_10MIN.csv'
df_ACCU_Intergrated = pd.read_csv(str_dirName_h + str_file_accu, index_col = 0)
df_ACCU_Intergrated

,YEAR,MONTH,DAY,HOUR,MINUTE,ELEC_ACTUAL_ACCU_EFF_561-1-1,ELEC_ACTUAL_ACCU_EFF_561-1-2,ELEC_ACTUAL_ACCU_EFF_561-1-3,ELEC_ACTUAL_ACCU_EFF_561-1-4,ELEC_ACTUAL_ACCU_EFF_561-2-1,...,ELEC_ACTUAL_ACCU_EFF_563-22-3,ELEC_ACTUAL_ACCU_EFF_563-22-4,ELEC_ACTUAL_ACCU_EFF_563-22-5,ELEC_ACTUAL_ACCU_EFF_563-22-6,ELEC_ACTUAL_ACCU_EFF_563-23-1,ELEC_ACTUAL_ACCU_EFF_563-23-2,ELEC_ACTUAL_ACCU_EFF_563-23-3,ELEC_ACTUAL_ACCU_EFF_563-23-4,ELEC_ACTUAL_ACCU_EFF_563-24-1,ELEC_ACTUAL_ACCU_EFF_563-24-2
0,2022,7,17,23,20,798.6900000000,650.6700000000,452.0000000000,812.2400000000,664.2700000000,...,524.5100000000,462.4000000000,489.8000000000,470.6400000000,358.2800000000,523.9700000000,441.8800000000,380.0800000000,393.9300000000,511.9200000000
1,2022,7,17,23,30,798.8600000000,650.8400000000,452.0400000000,812.5300000000,664.4500000000,...,524.6600000000,462.5100000000,489.8300000000,470.6800000000,358.3300000000,524.0500000000,441.9400000000,380.1400000000,393.9600000000,512.0200000000
2,2022,7,17,23,40,799.0200000000,651.0200000000,452.0700000000,812.7400000000,664.5800000000,...,524.7800000000,462.6100000000,489.8500000000,470.7200000000,358.3900000000,524.1100000000,441.9900000000,NaN,393.9800000000,512.1100000000
3,2022,7,17,23,50,799.1600000000,651.2500000000,452.1000000000,812.9800000000,664.7200000000,...,524.8800000000,462.7100000000,489.8900000000,470.7500000000,358.4400000000,524.1700000000,442.0400000000,380.2400000000,394.0100000000,512.2000000000
4,2022,7,18,0,0,799.3000000000,651.4600000000,452.1200000000,813.2500000000,664.8700000000,...,525.0000000000,462.8100000000,489.9300000000,470.7900000000,358.4800000000,524.2300000000,442.0800000000,380.2900000000,394.0400000000,512.2800000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99165,2024,6,5,14,50,10755.0000000000,9350.7800000000,7648.3000000000,10798.9000000000,11286.6500000000,...,NaN,NaN,7520.4100000000,NaN,6486.7300000000,NaN,NaN,NaN,NaN,NaN
99166,2024,6,5,15,0,10755.0400000000,9350.8400000000,7648.5300000000,10798.9300000000,11286.7500000000,...,NaN,NaN,7520.4500000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99167,2024,6,5,15,10,10755.0700000000,9350.9000000000,7648.8700000000,10798.9600000000,11286.8200000000,...,NaN,NaN,7520.5300000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99168,2024,6,5,15,20,10755.1200000000,9350.9700000000,7649.1600000000,10798.9800000000,11286.8900000000,...,NaN,6405.4600000000,7520.5800000000,NaN,6486.7800000000,NaN,6719.9600000000,NaN,NaN,7788.9200000000


## 04. 순시사용량

In [110]:
## 모든 호실의 사용량을 변수화하여 한 데이터셋에 Combine
df_kier_h_Combined = df_ACCU_Intergrated[['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE']]
list_HID = df_kier_hList['HOUSE_ID'].drop_duplicates()
for house in list_HID:
    print(str(house) + " H")

    str_col_accu_h, str_col_inst_h = str_col_accu + "_" + house, str_col_inst + "_" + house
    str_file_inst = 'KIER_' + str_domain + '_' + str(house) + '_INST_Raw.csv'

    df_h_tmp = df_ACCU_Intergrated[['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE', str_col_accu_h]]
    df_h_tmp[str_col_inst_h] = 0

    for i in range(0, len(df_h_tmp[str_col_inst_h]) - 1) : df_h_tmp[str_col_inst_h].iloc[i] = df_h_tmp[str_col_accu_h].iloc[i + 1] - df_h_tmp[str_col_accu_h].iloc[i]

    df_kier_h_Combined = pd.merge(df_kier_h_Combined, df_h_tmp[['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE', str_col_inst_h]]
                                  , how = 'left', on = ['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE'])

str_file = 'KIER_' + str_domain + '_INST_10MIN.csv'       
df_kier_h_Combined.to_csv(str_dirName_h + str_file)
print(df_kier_h_Combined.shape, ' /// ', df_kier_h_Combined.columns)
df_kier_h_Combined

561-1-1 H
561-1-2 H
561-1-3 H
561-1-4 H
561-2-1 H
561-2-2 H
561-2-3 H
561-2-4 H
561-3-1 H
561-3-2 H
561-3-3 H
561-3-4 H
561-4-1 H
561-4-2 H
561-4-3 H
561-4-4 H
561-5-1 H
561-5-2 H
561-5-3 H
561-5-4 H
561-6-1 H
561-6-2 H
561-6-3 H
561-6-4 H
561-7-1 H
561-7-2 H
561-7-3 H
561-7-4 H
561-8-1 H
561-8-2 H
561-8-3 H
561-8-4 H
561-9-1 H
561-9-2 H
561-9-3 H
561-9-4 H
561-10-1 H
561-10-2 H
561-10-3 H
561-10-4 H
561-11-1 H
561-11-2 H
561-11-3 H
561-11-4 H
561-12-1 H
561-12-2 H
561-12-3 H
561-12-4 H
561-13-1 H
561-13-2 H
561-13-3 H
561-13-4 H
561-14-1 H
561-14-2 H
561-14-3 H
561-14-4 H
561-15-1 H
561-15-2 H
561-15-3 H
561-15-4 H
561-16-1 H
561-16-2 H
561-16-3 H
561-16-4 H
561-17-1 H
561-17-2 H
561-17-3 H
561-17-4 H
561-18-1 H
561-18-2 H
561-18-3 H
561-18-4 H
562-1-1 H
562-1-2 H
562-1-3 H
562-1-4 H
562-1-5 H
562-1-6 H
562-2-1 H
562-2-2 H
562-2-3 H
562-2-4 H
562-2-5 H
562-2-6 H
562-3-1 H
562-3-2 H
562-3-3 H
562-3-4 H
562-3-5 H
562-3-6 H
562-4-1 H
562-4-2 H
562-4-3 H
562-4-4 H
562-4-5 H
562-4-6 H
562-

,YEAR,MONTH,DAY,HOUR,MINUTE,ELEC_INST_EFF_561-1-1,ELEC_INST_EFF_561-1-2,ELEC_INST_EFF_561-1-3,ELEC_INST_EFF_561-1-4,ELEC_INST_EFF_561-2-1,...,ELEC_INST_EFF_563-22-3,ELEC_INST_EFF_563-22-4,ELEC_INST_EFF_563-22-5,ELEC_INST_EFF_563-22-6,ELEC_INST_EFF_563-23-1,ELEC_INST_EFF_563-23-2,ELEC_INST_EFF_563-23-3,ELEC_INST_EFF_563-23-4,ELEC_INST_EFF_563-24-1,ELEC_INST_EFF_563-24-2
0,2022,7,17,23,20,0.1700000000,0.1700000000,0.0400000000,0.2900000000,0.1800000000,...,0.1500000000,0.1100000000,0.0300000000,0.0400000000,0.0500000000,0.0800000000,0.0600000000,0.0600000000,0.0300000000,0.1000000000
1,2022,7,17,23,30,0.1600000000,0.1800000000,0.0300000000,0.2100000000,0.1300000000,...,0.1200000000,0.1000000000,0.0200000000,0.0400000000,0.0600000000,0.0600000000,0.0500000000,NaN,0.0200000000,0.0900000000
2,2022,7,17,23,40,0.1400000000,0.2300000000,0.0300000000,0.2400000000,0.1400000000,...,0.1000000000,0.1000000000,0.0400000000,0.0300000000,0.0500000000,0.0600000000,0.0500000000,NaN,0.0300000000,0.0900000000
3,2022,7,17,23,50,0.1400000000,0.2100000000,0.0200000000,0.2700000000,0.1500000000,...,0.1200000000,0.1000000000,0.0400000000,0.0400000000,0.0400000000,0.0600000000,0.0400000000,0.0500000000,0.0300000000,0.0800000000
4,2022,7,18,0,0,0.1500000000,0.2100000000,0.0300000000,0.2200000000,0.1400000000,...,0.1100000000,0.1100000000,0.0400000000,0.0400000000,0.0300000000,0.0600000000,0.0600000000,0.0300000000,0.0200000000,0.0600000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99165,2024,6,5,14,50,0.0400000000,0.0600000000,0.2300000000,0.0300000000,0.1000000000,...,NaN,NaN,0.0400000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99166,2024,6,5,15,0,0.0300000000,0.0600000000,0.3400000000,0.0300000000,0.0700000000,...,NaN,NaN,0.0800000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99167,2024,6,5,15,10,0.0500000000,0.0700000000,0.2900000000,0.0200000000,0.0700000000,...,NaN,NaN,0.0500000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99168,2024,6,5,15,20,0.0500000000,0.1000000000,0.1300000000,0.0200000000,0.0800000000,...,NaN,NaN,0.0900000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 음수값 제거

In [111]:
## Re-Load Dataset
str_file = 'KIER_' + str_domain + '_INST_10MIN.csv'       
df_kier_h_Combined = pd.read_csv(str_dirName_h + str_file, index_col = 0)

In [112]:
list_col_tar = list(df_kier_h_Combined.columns)[5:]
list_col_tar

['ELEC_INST_EFF_561-1-1',
 'ELEC_INST_EFF_561-1-2',
 'ELEC_INST_EFF_561-1-3',
 'ELEC_INST_EFF_561-1-4',
 'ELEC_INST_EFF_561-2-1',
 'ELEC_INST_EFF_561-2-2',
 'ELEC_INST_EFF_561-2-3',
 'ELEC_INST_EFF_561-2-4',
 'ELEC_INST_EFF_561-3-1',
 'ELEC_INST_EFF_561-3-2',
 'ELEC_INST_EFF_561-3-3',
 'ELEC_INST_EFF_561-3-4',
 'ELEC_INST_EFF_561-4-1',
 'ELEC_INST_EFF_561-4-2',
 'ELEC_INST_EFF_561-4-3',
 'ELEC_INST_EFF_561-4-4',
 'ELEC_INST_EFF_561-5-1',
 'ELEC_INST_EFF_561-5-2',
 'ELEC_INST_EFF_561-5-3',
 'ELEC_INST_EFF_561-5-4',
 'ELEC_INST_EFF_561-6-1',
 'ELEC_INST_EFF_561-6-2',
 'ELEC_INST_EFF_561-6-3',
 'ELEC_INST_EFF_561-6-4',
 'ELEC_INST_EFF_561-7-1',
 'ELEC_INST_EFF_561-7-2',
 'ELEC_INST_EFF_561-7-3',
 'ELEC_INST_EFF_561-7-4',
 'ELEC_INST_EFF_561-8-1',
 'ELEC_INST_EFF_561-8-2',
 'ELEC_INST_EFF_561-8-3',
 'ELEC_INST_EFF_561-8-4',
 'ELEC_INST_EFF_561-9-1',
 'ELEC_INST_EFF_561-9-2',
 'ELEC_INST_EFF_561-9-3',
 'ELEC_INST_EFF_561-9-4',
 'ELEC_INST_EFF_561-10-1',
 'ELEC_INST_EFF_561-10-2',
 'ELEC_INS

In [113]:
## 음수값 제거
cnt_minus = 0
for col_tar in list_col_tar:
    for i in range(0, len(df_kier_h_Combined[list_col_tar])):
        if df_kier_h_Combined[col_tar].loc[i] < 0 : 
            print(i , ' /// ', df_kier_h_Combined[col_tar].iloc[i])
            df_kier_h_Combined[col_tar].iloc[i] = np.nan
            cnt_minus = cnt_minus + 1

print(cnt_minus)

1066  ///  -0.0099999999999909
1068  ///  -0.0099999999999909
1073  ///  -0.0099999999999909
1078  ///  -0.0099999999999909
1090  ///  -0.0099999999999909
1214  ///  -0.0099999999999909
1216  ///  -0.0099999999999909
1221  ///  -0.0199999999999818
1226  ///  -0.0099999999999909
1502  ///  -0.0099999999999909
1651  ///  -0.0099999999999909
1653  ///  -0.0100000000001045
2230  ///  -0.0099999999999909
3658  ///  -0.0099999999999909
3664  ///  -0.0099999999999909
3665  ///  -0.0099999999999909
3666  ///  -0.0099999999999909
3668  ///  -0.0099999999999909
3963  ///  -0.0100000000001045
3967  ///  -0.0099999999999909
4238  ///  -0.0099999999999909
4242  ///  -0.0099999999999909
4249  ///  -0.0099999999999909
4250  ///  -0.0099999999999909
4251  ///  -0.0099999999999909
4252  ///  -0.0099999999999909
4253  ///  -0.0099999999999909
4254  ///  -0.0099999999999909
4255  ///  -0.0099999999999909
4521  ///  -0.0099999999999909
4530  ///  -0.0099999999999909
4536  ///  -0.0099999999999909
4815  //

In [114]:
## Export Datasets
str_file = 'KIER_' + str_domain + '_INST_10MIN.csv'       
df_kier_h_Combined.to_csv(str_dirName_h + str_file)

#### 2024-07-30 기준 Domain별 결측치 수 
- ELEC : 3799
- HEAT : 2205
- WATER : 7
- HOT_FLOW : 60
- HOT_HEAT : 36
- GAS : 2662

In [115]:
# ## [미사용] Interpolation 후 수행해야할 연산
# list_col = df_kier_h_Combined.columns[5:]
# print(list_col)
# df_kier_Calc = df_kier_h_Combined[list_col]

# ## 사용량 평균 및 합계 Column 추가
# df_kier_h_Combined['MEAN_OF_INST'], df_kier_h_Combined['SUM_OF_INST'] = df_kier_Calc.mean(axis = 1), df_kier_Calc.sum(axis = 1)

# str_file = 'KIER_' + str_domain + '_INST_10MIN.csv'       
# df_kier_h_Combined.to_csv(str_dirName_h + str_file)
# print(df_kier_h_Combined.isnull().sum())
# df_kier_h_Combined